In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#virgulden sonra gösterilecek olan sayı sayısı
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import matplotlib.pyplot as plt

In [4]:
df_2010_2011 = pd.read_excel("../input/uci-online-retail-ii-data-set/online_retail_II.xlsx", sheet_name = "Year 2010-2011")
                             

In [11]:
df = df_2010_2011.copy()

In [12]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom


In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom


In [21]:
#df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

In [23]:
df["TotalPrice"] = df["Quantity"]*df["Price"]

In [13]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [14]:
df.dropna(inplace = True)

In [15]:
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,406830,12,249,-80995,-2,1,1,2,5,12,24,36,120,80995
Price,406830,3,69,0,0,0,1,1,2,4,7,8,15,38970
Customer ID,406830,15288,1714,12346,12415,12626,12876,13953,15152,16791,17719,17905,18212,18287


In [24]:
for feature in ["Quantity","Price","TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Quantity yes
948
Price yes
846
TotalPrice yes
1030


In [25]:
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [26]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [27]:
import datetime as dt
today_date = dt.datetime(2011,12,9)

In [30]:
df.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

,InvoiceDate
Customer ID,
12346,2011-01-18 10:17:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00


In [31]:
df.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

,InvoiceDate
Customer ID,
12346,2011-01-18 10:17:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00


In [32]:
df["Customer ID"] = df["Customer ID"].astype(int)

In [33]:
(today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"})).head()

,InvoiceDate
Customer ID,
12346,324 days 13:43:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [34]:
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))

In [35]:
temp_df.rename(columns={"InvoiceDate": "Recency"}, inplace = True)

In [36]:
temp_df.head()


,Recency
Customer ID,
12346,324 days 13:43:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [37]:
recency_df = temp_df["Recency"].apply(lambda x: x.days)

In [38]:
recency_df.head()

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
Name: Recency, dtype: int64

In [39]:
temp_df = df.groupby(["Customer ID","Invoice"]).agg({"Invoice":"count"})

In [40]:
temp_df.groupby("Customer ID").agg({"Invoice":"count"}).head()

,Invoice
Customer ID,
12346,2
12347,7
12348,4
12349,1
12350,1


In [41]:
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"count"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)
freq_df.head()

,Frequency
Customer ID,
12346,2
12347,7
12348,4
12349,1
12350,1


In [42]:
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})

In [43]:
monetary_df.head()

,TotalPrice
Customer ID,
12346,0
12347,4310
12348,1797
12349,1758
12350,334


In [44]:
monetary_df.rename(columns={"TotalPrice": "Monetary"}, inplace = True)

In [45]:
print(recency_df.shape,freq_df.shape,monetary_df.shape)

(4372,) (4372, 1) (4372, 1)


In [46]:
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)

In [47]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,2,0
12347,1,7,4310
12348,74,4,1797
12349,17,1,1758
12350,309,1,334


In [48]:
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])

In [59]:
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])

ValueError: Bin labels must be one fewer than the number of bin edges

In [49]:
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

In [50]:
rfm.head()


,Recency,Frequency,Monetary,RecencyScore,MonetaryScore
Customer ID,,,,,
12346,324,2,0,1,1
12347,1,7,4310,5,5
12348,74,4,1797,2,4
12349,17,1,1758,4,4
12350,309,1,334,1,2


In [52]:
(rfm['RecencyScore'].astype(str) + 
 rfm['FrequencyScore'].astype(str) + 
 rfm['MonetaryScore'].astype(str)).head()

KeyError: 'FrequencyScore'

In [53]:
rfm["RFM_SCORE"] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)

KeyError: 'FrequencyScore'

In [54]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,MonetaryScore
Customer ID,,,,,
12346,324,2,0,1,1
12347,1,7,4310,5,5
12348,74,4,1797,2,4
12349,17,1,1758,4,4
12350,309,1,334,1,2


In [ ]:
rfm.describe().T

In [ ]:
rfm[rfm["RFM_SCORE"] == "555"].head()

In [ ]:
rfm[rfm["RFM_SCORE"] == "111"].head()

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [ ]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

In [ ]:
rfm[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

In [ ]:
rfm[rfm["Segment"] == "Need Attention"].head()

In [ ]:
rfm[rfm["Segment"] == "New Customers"].index

In [ ]:
new_df = pd.DataFrame()
new_df["NewCustomerID"] = rfm[rfm["Segment"] == "New Customers"].index

In [ ]:
new_df.head()

In [9]:
new_df.to_csv("new_customers.csv")

NameError: name 'new_df' is not defined